In [ ]:
#@title <b><-- Convert TavernAI characters to SillyTavern format</b>

!mkdir /convert
%cd /convert

import os
from google.colab import drive

drive.mount("/convert/drive")

!git clone -b tools https://github.com/EnergoStalin/SillyTavern.git
%cd SillyTavern

!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0

%cd tools/charaverter

!npm i

path = "/convert/drive/MyDrive/TavernAI/characters"
output = "/convert/drive/MyDrive/SillyTavern/characters"
if not os.path.exists(path):
  path = output

!mkdir -p $output
!node main.mjs $path $output

drive.flush_and_unmount()

%cd /
!rm -rf /convert

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldAI below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b><-- Select your model below and then click this to start KoboldAI</b>

Model = "Pygmalion 6B" #@param ["Nerys V2 6B", "Erebus 6B", "Skein 6B", "Janeway 6B", "Adventure 6B", "Pygmalion 6B", "Pygmalion 6B Dev", "Lit V2 6B", "Lit 6B", "Shinen 6B", "Nerys 2.7B", "AID 2.7B", "Erebus 2.7B", "Janeway 2.7B", "Picard 2.7B", "Horni LN 2.7B", "Horni 2.7B", "Shinen 2.7B", "OPT 2.7B", "Fairseq Dense 2.7B", "Neo 2.7B", "Pygway 6B", "Nerybus 6.7B", "Pygway v8p4", "PPO-Janeway 6B", "PPO Shygmalion 6B", "LLaMA 7B", "Janin-GPTJ", "Javelin-GPTJ", "Javelin-R", "Janin-R", "Javalion-R", "Javalion-GPTJ", "Javelion-6B", "GPT-J-Pyg-PPO-6B", "ppo_hh_pythia-6B", "ppo_hh_gpt-j", "GPT-J-Pyg_PPO-6B", "GPT-J-Pyg_PPO-6B-Dev-V8p4", "Dolly_GPT-J-6b", "Dolly_Pyg-6B"] {allow-input: true}
Version = "Official" #@param ["Official", "United"] {allow-input: true}
Provider = "Localtunnel" #@param ["Localtunnel"]
ForceInitSteps = [] #@param {allow-input: true}
UseGoogleDrive = True #@param {type:"boolean"}
StartKoboldAI = True #@param {type:"boolean"}
ModelsFromDrive = False #@param {type:"boolean"}

%cd /content

!cat .ii
!nvidia-smi

import os, subprocess, time, pathlib, json, base64

class ModelData:
  def __init__(self, name, version = Version, revision = "", path = "", download = ""):
    try:
      self.name = base64.b64decode(name.encode("ascii")).decode("ascii")
    except:
      self.name = name
    self.version = version 
    self.revision = revision
    self.path = path
    self.download = download
  def args(self):
    args = ["-m", self.name, "-g", self.version]
    if(self.revision):
      args += ["-r", self.revision]
    return args

class IncrementialInstall:
  def __init__(self, tasks = [], force = []):
    self.tasks = tasks
    self.completed = list(filter(lambda x: not x in force, self.__completed()))

  def __completed(self):
    try:
      with open(".ii") as f:
        return json.load(f)
    except:
      return []

  def addTask(self, name, func):
    self.tasks.append({"name": name, "func": func})

  def run(self):
    todo = list(filter(lambda x: not x["name"] in self.completed, self.tasks))
    try:
      for task in todo:
        task["func"]()
        self.completed.append(task["name"])
    finally:
      with open(".ii", "w") as f:
        json.dump(self.completed, f)

ii = IncrementialInstall(force=ForceInitSteps)

def create_paths(paths):
  for directory in paths:
    if not os.path.exists(directory):
      os.makedirs(directory)

# link source to dest copying dest to source if not present first
def link(srcDir, destDir, files):
  for file in files:
    source = os.path.join(srcDir, file)
    dest = os.path.join(destDir, file)
    if not os.path.exists(source):
      !cp -r "$dest" "$source"
    !rm -rf "$dest"
    !ln -fs "$source" "$dest"

from google.colab import drive
if UseGoogleDrive:
  drive.mount("/content/drive/")
else:
  create_paths([
      "/content/drive/MyDrive"
  ])

model = {
  "Nerys V2 6B": ModelData("KoboldAI/OPT-6B-nerys-v2"),
  "Erebus 6B": ModelData("KoboldAI/OPT-6.7B-Erebus"),
  "Skein 6B": ModelData("KoboldAI/GPT-J-6B-Skein"),
  "Janeway 6B": ModelData("KoboldAI/GPT-J-6B-Janeway"),
  "Adventure 6B": ModelData("KoboldAI/GPT-J-6B-Adventure"),
  "UHlnbWFsaW9uIDZC": ModelData("UHlnbWFsaW9uQUkvcHlnbWFsaW9uLTZi"),
  "UHlnbWFsaW9uIDZCIERldg==": ModelData("UHlnbWFsaW9uQUkvcHlnbWFsaW9uLTZi", revision = "dev"),
  "Lit V2 6B": ModelData("hakurei/litv2-6B-rev3"),
  "Lit 6B": ModelData("hakurei/lit-6B"),
  "Shinen 6B": ModelData("KoboldAI/GPT-J-6B-Shinen"),
  "Nerys 2.7B": ModelData("KoboldAI/fairseq-dense-2.7B-Nerys"),
  "Erebus 2.7B": ModelData("KoboldAI/OPT-2.7B-Erebus"),
  "Janeway 2.7B": ModelData("KoboldAI/GPT-Neo-2.7B-Janeway"),
  "Picard 2.7B": ModelData("KoboldAI/GPT-Neo-2.7B-Picard"),
  "AID 2.7B": ModelData("KoboldAI/GPT-Neo-2.7B-AID"),
  "Horni LN 2.7B": ModelData("KoboldAI/GPT-Neo-2.7B-Horni-LN"),
  "Horni 2.7B": ModelData("KoboldAI/GPT-Neo-2.7B-Horni"),
  "Shinen 2.7B": ModelData("KoboldAI/GPT-Neo-2.7B-Shinen"),
  "Fairseq Dense 2.7B": ModelData("KoboldAI/fairseq-dense-2.7B"),
  "OPT 2.7B": ModelData("facebook/opt-2.7b"),
  "Neo 2.7B": ModelData("EleutherAI/gpt-neo-2.7B"),
  "Pygway 6B": ModelData("TehVenom/PPO_Pygway-6b"),
  "Nerybus 6.7B": ModelData("KoboldAI/OPT-6.7B-Nerybus-Mix"),
  "Pygway v8p4": ModelData("TehVenom/PPO_Pygway-V8p4_Dev-6b"),
  "PPO-Janeway 6B": ModelData("TehVenom/PPO_Janeway-6b"),
  "PPO Shygmalion 6B": ModelData("TehVenom/PPO_Shygmalion-6b"),
  "LLaMA 7B": ModelData("decapoda-research/llama-7b-hf"),
  "Janin-GPTJ": ModelData("digitous/Janin-GPTJ"),
  "Javelin-GPTJ": ModelData("digitous/Javelin-GPTJ"),
  "Javelin-R": ModelData("digitous/Javelin-R"),
  "Janin-R": ModelData("digitous/Janin-R"),
  "Javalion-R": ModelData("digitous/Javalion-R"),
  "Javalion-GPTJ": ModelData("digitous/Javalion-GPTJ"),
  "Javelion-6B": ModelData("Cohee/Javelion-6b"),
  "GPT-J-Pyg-PPO-6B": ModelData("TehVenom/GPT-J-Pyg_PPO-6B"),
  "ppo_hh_pythia-6B": ModelData("reciprocate/ppo_hh_pythia-6B"),
  "ppo_hh_gpt-j": ModelData("reciprocate/ppo_hh_gpt-j"),
  "Alpaca-7B": ModelData("chainyo/alpaca-lora-7b"),
  "LLaMA 4-bit": ModelData("decapoda-research/llama-13b-hf-int4"),
  "GPT-J-Pyg_PPO-6B": ModelData("TehVenom/GPT-J-Pyg_PPO-6B"),
  "GPT-J-Pyg_PPO-6B-Dev-V8p4": ModelData("TehVenom/GPT-J-Pyg_PPO-6B-Dev-V8p4"),
  "Dolly_GPT-J-6b": ModelData("TehVenom/Dolly_GPT-J-6b"),
  "Dolly_Pyg-6B": ModelData("TehVenom/AvgMerge_Dolly-Pygmalion-6b")
}
model = model.get(Model, model)
model = model.get(base64.b64encode(Model.encode("ascii")).decode("ascii"), ModelData(Model, Version))

if StartKoboldAI:
  def downloadKobold():
    !wget https://koboldai.org/ckds && chmod +x ckds
  def initKobold():
    !./ckds --init only

  ii.addTask("Download KoboldAI", downloadKobold)
  ii.addTask("Init KoboldAI", initKobold)
  
  ii.run()

kargs = ["/content/ckds"]
if not ModelsFromDrive:
  kargs += ["-x", "colab", "-l", "colab"]
if Provider == "Localtunnel":
  kargs += ["--localtunnel", "yes"]

kargs += model.args()

url = ""
print(kargs)

if StartKoboldAI:
  p = subprocess.Popen(kargs, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

  prefix = "KoboldAI has finished loading and is available at the following link"
  urlprefix = f"{prefix}: "
  ui1prefix = f"{prefix} for UI 1: "
  while True:
      line = p.stdout.readline().decode().strip()
      if urlprefix in line:
          url = line.split(urlprefix)[1]
          break
      elif ui1prefix in line:
          url = line.split(ui1prefix)[1]
          break
      elif not line:
          break
      print(line)
      if "INIT" in line and "Transformers" in line:
        print("Model loading... (It will take 2 - 5 minutes)")

print(url)

# #TavernAI
%cd /
def setupNVM():
  !curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
ii.addTask("Setup NVM", setupNVM)

def installNode():
  !nvm install 19.1.0
  !nvm use 19.1.0
ii.addTask("Install node", installNode)

def cloneTavern():
  !git clone https://github.com/EnergoStalin/SillyTavern
ii.addTask("Clone SillyTavern", cloneTavern)

ii.run()
%cd /SillyTavern

if UseGoogleDrive:
  %env googledrive=2

  def setupTavernPaths():
    tdrive = "/content/drive/MyDrive/SillyTavern"
    create_paths([
        tdrive,
        os.path.join("public", "groups"),
        os.path.join("public", "group chats")
    ])
    link(tdrive, "public", [
        "settings.json",
        "backgrounds",
        "characters",
        "chats",
        "User Avatars",
        "css",
        "worlds",
        "group chats",
        "groups",
    ])
  ii.addTask("Setup Tavern Paths", setupTavernPaths)

def installTavernDependencies():
  !npm install
  !npm install -g localtunnel
ii.addTask("Install Tavern Dependencies", installTavernDependencies)
ii.run()

%env colaburl=$url
%env SILLY_TAVERN_PORT=5001
print("KoboldAI LINK:", url, "###SillyTavern LINK###", sep="\n")
p = subprocess.Popen(["lt", "--port", "5001"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(p.stdout.readline().decode().strip())
!node server.js